# Aanbod huurwoningen in Nederland

In [64]:
import pandas as pd

pararius = pd.read_csv("../data/pararius_listings.csv")
pararius.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Link                2007 non-null   object 
 1   Huurprijs           1800 non-null   object 
 2   Locatie             1800 non-null   object 
 3   m2                  2005 non-null   object 
 4   Kamers              2005 non-null   object 
 5   Interieur           1702 non-null   object 
 6   Huurovereenkomst    883 non-null    object 
 7   Type woning         0 non-null      float64
 8   Bouwjaar            1689 non-null   object 
 9   Badkamers           1374 non-null   float64
 10  Faciliteiten        1122 non-null   object 
 11  Balkon              1606 non-null   object 
 12  Tuin                1507 non-null   object 
 13  Omschrijving tuin   241 non-null    object 
 14  Energie label       1454 non-null   object 
 15  Opslag              1070 non-null   object 
 16  Parker

### Opschonen

Om de dataset te kunnen analyseren moeten eerst een aantal kolommen een ander datatype krijgen, worden opgeschoond en waar nodig worden bijgevuld:

1. Er zijn twee links die geen waarden hebben voor `Huurprijs`, `Locatie` en `Beschrijving`. Deze links werken niet meer dus we verwijderen deze rijen.
2. De `Huurprijs` kolom moet worden omgezet naar een `float` datatype.
    - Sommige van deze waarden worden als volgt weergeven: '€ 1.075 - 1.775 per month'. In dat geval berekenen we het gemiddelde van de prijs.
    - Voor sommige huurwoningen moet je de prijs opvragen, de waarde is dan 'Price on request'. Voor dat geval voegen we een nieuwe kolom toe `Verzoek` met als waarde `True` of `False`.
3. De `m2` moet ook worden omgezet naar `float`.
    - Hier geldt dezelfde uitzondering als bij de huurprijs. We berekenen ook het gemiddelde aantal vierkante meter in dit geval.
4. De `Kamers` kolom moet alleen een cijfer bevatten en worden omgezet naar `int` datatype.
5. De `Bouwjaar` kolom moet worden omgezet naar `int` datatype.
6. De `Locatie` kolom bevat een postcode en de naam van de buurt. Dit gaan we opsplitsen in twee kolommen.
7. De URL in de `Link` kolom bevat de gemeente waarin de woning staat. Hiervoor maken we een nieuwe kolom `Gemeente`.

In [54]:
def schoon_huurprijs(prijs):
    """Functie om de 'Huurprijs' kolom op te schonen."""
    if isinstance(prijs, float) or prijs is None:
        return prijs
    
    if "Price on request" in str(prijs):
        return None # Lege waarde voor de huurprijs
    
    prijs = prijs.replace("€", "").replace("per month", "").replace(",", "").strip()

    # Check of de huurprijs twee prijzen kan zijn
    if "-" in prijs:
        laag, hoog = prijs.split("-")
        laag = float(laag.strip())
        hoog = float(hoog.strip())
        return (laag + hoog) / 2 # Bereken het gemiddelde van de twee prijzen
    else:
        return float(prijs)


def schoon_m2(waarde):
    """Functie om de 'm2' kolom op te schonen."""
    if pd.isnull(waarde):
        return None
    
    waarde = waarde.replace("m²", "").replace("㎡", "").strip()

    # Bereken het gemiddelde als er twee waarden voor 'm2' zijn
    if "-" in waarde:
        laag, hoog = waarde.split("-")
        laag = laag.strip()
        hoog = hoog.strip()
        return (float(laag) + float(hoog)) / 2
    else:
        return float(waarde)


def schoon_kamers(waarde):
    """Functie om de 'Kamers' kolom op te schonen."""
    return int(waarde.replace("rooms", "").replace("room", "").strip())


def krijg_buurt(locatie):
    """Functie die de 'Locatie' kolom opsplits in de postcode en buurt."""
    if pd.isnull(locatie):
        return None
    
    if "(" in locatie and ")" in locatie:
        buurt = locatie.split("(")[-1].split(")")[0].strip()
        return buurt
    else:
        return None


def krijg_gemeente(link):
    """Functie die de gemeentenaam uit de URL in de 'Link' kolom haalt."""
    if pd.isnull(link):
        return None
    
    # Splits de URL op en haal de naam van de stad op
    try:
        return link.split("/")[4] #
    except IndexError:
        return None

In [55]:
# Vind de lege rijen en verwijder ze
lege_rijen = pararius[pararius[['Huurprijs', 'Locatie', 'Beschrijving']].isnull().all(axis=1)]
pararius = pararius.drop(lege_rijen.index)

# Huurprijs opschonen
pararius["Verzoek"] = pararius["Huurprijs"].apply(lambda x: True if "Price on request" in str(x) else False)
pararius["Huurprijs"] = pararius["Huurprijs"].apply(schoon_huurprijs)

# Vierkante meter opschonen
pararius["m2"] = pararius["m2"].apply(schoon_m2)

# Kamers opschonen
pararius["Kamers"] = pararius["Kamers"].apply(schoon_kamers)

# Bouwjaar omzetten naar integer
pararius["Bouwjaar"] = pd.to_numeric(pararius["Bouwjaar"], errors='coerce')

# Locatie opschonen
pararius['Buurt'] = pararius['Locatie'].apply(krijg_buurt)
pararius['Locatie'] = pararius['Locatie'].apply(lambda x: x.split('(')[0].strip() if pd.notnull(x) else x)

# Voeg de 'Stad' kolom toe
pararius["Gemeente"] = pararius["Link"].apply(krijg_gemeente)

pararius.head()

,Link,Huurprijs,Locatie,m2,Kamers,Interieur,Huurovereenkomst,Type woning,Bouwjaar,Badkamers,...,Omschrijving tuin,Energie label,Opslag,Parkeren,Type parkeerplaats,Garage,Beschrijving,Verzoek,Buurt,Gemeente
0,https://www.pararius.com/apartment-for-rent/am...,1900.0,1012 ES,65.0,2,Furnished,Unlimited period,NaN,1950.0,1.0,...,NaN,NaN,Not present,No,NaN,No,"Description\r\nGreat Location , 1 bedroom apt ...",False,Burgwallen-Oude Zijde,amsterdam
1,https://www.pararius.com/apartment-for-rent/zo...,2100.0,2718 SJ,106.0,3,Upholstered,Unlimited period,NaN,1996.0,1.0,...,NaN,A+,Not present,Yes,Public,No,Description\r\nUnfurnished 4-room apartment lo...,False,Lansinghage c.a.,zoetermeer
2,https://www.pararius.com/apartment-for-rent/de...,800.0,2563 BH,35.0,2,Upholstered,Unlimited period,NaN,1900.0,1.0,...,NaN,NaN,Not present,Yes,Permit,No,Description\r\nSuper nice apartment on Laan va...,False,Valkenboskwartier,den-haag
3,https://www.pararius.com/apartment-for-rent/ti...,1725.0,5038 BW,84.0,2,Furnished,Temporary rental,NaN,2007.0,1.0,...,NaN,A,Not present,Yes,Garage,Yes,Description\r\nCan be rented for a maximum of ...,False,Binnenstad Oost,tilburg
6,https://www.pararius.com/house-for-rent/noordw...,1950.0,2201 WH,113.0,5,Furnished,Unlimited period,NaN,1986.0,1.0,...,NaN,B,Not present,Yes,Public,No,Description\r\nFor Rent: Temporary Home in Noo...,False,Vinkeveld Zuid,noordwijk-zh


## 1. Middenhuurwoningen

Op 25 juni 2024 heeft de Eerste Kamer ingestemd met de [Wet betaalbare huur](https://www.vbk.nl/legalupdate/wet-betaalbare-huur-middenhuur-aangenomen). Dit betekent o.a. dat de huurprijzen van woningen t/m 186 punten (middenhuurwoningen) begrensd worden. Het middenhuursegment gaat dus bestaan uit woningen met een huurprijs van meer dan € 879,66 per maand en maximaal € 1.165,81.

We gaan dus woningen filteren tussen deze prijsgrenzen. Hoeveel woningen uit de gehele dataset behoren tot dit middenhuursegment?


In [56]:
# Filter het middenhuursegment
middenhuur = pararius[(pararius['Huurprijs'] > 879.66) & (pararius['Huurprijs'] < 1165.81)]

rijen = len(pararius)
middenhuur_rijen = len(middenhuur)

percentage_middenhuur = (middenhuur_rijen / rijen) * 100

print(f"Percentage huurwoningen die tot het middenhuursegment horen: {percentage_middenhuur:.2f}%")

Percentage huurwoningen die tot het middenhuursegment horen: 9.43%


Wat is de gemiddelde prijs en aantal vierkante meter voor een middenhuurwoning?

In [57]:
# Bereken de gemiddelden
gem_huurprijs = middenhuur["Huurprijs"].mean()
gem_m2 = middenhuur["m2"].mean()

print(f"Gemiddelde huurprijs voor een middenhuurwoning: €{gem_huurprijs:.2f}")
print(f"Gemiddeld aantal vierkante meter voor een middenhuurwoning: {gem_m2:.2f} m²")

Gemiddelde huurprijs voor een middenhuurwoning: €1016.73
Gemiddeld aantal vierkante meter voor een middenhuurwoning: 60.53 m²


In welke gemeenten staan de meeste middenhuurwoningen?

In [58]:
top_5_gemeenten= middenhuur["Gemeente"].value_counts().head(5)

print(top_5_gemeenten)

Gemeente
groningen     18
rotterdam     10
maastricht     7
nijmegen       5
utrecht        5
Name: count, dtype: int64


## 2. WOZ-waarde

In [63]:
buurtcodes = pd.read_csv("../data/raw/buurtcodes.csv")
woz_waarden = pd.read_csv("../data/raw/woz_waarden.csv")

# Voeg de buurtnaam toe aan de dataframe met WOZ-waarden
merged_df = woz_waarden.merge(buurtcodes[["Key", "Title"]], on="Key", how="left")

woz_waarden["Key"] = merged_df["Title"]
woz_waarden = merged_df.drop(columns=['ID'])

woz_waarden.head()


,Key,Gemiddelde WOZ,Meest voorkomende postcode,Title
0,NL00,368.0,.,Nederland
1,GM1680,343.0,.,Aa en Hunze
2,WK168000,342.0,.,Wijk 00 Annen
3,BU16800000,333.0,9468,Annen
4,BU16800009,579.0,9468,Verspreide huizen Annen


### Opschonen

In [60]:
# Alle spaties/witregels strippen
woz_waarden = woz_waarden.map(lambda x: x.strip() if isinstance(x, str) else x)

# Missende waarden opvullen
woz_waarden['Meest voorkomende postcode'] = woz_waarden['Meest voorkomende postcode'].replace('.', '')



Meest voorkomende postcode
        3791
5571      26
7731      24
2411      24
9403      23
        ... 
8361       1
8025       1
8035       1
8033       1
9444       1
Name: count, Length: 3948, dtype: int64


## 3. Punten toekennen

Het [woningwaarderingsstelsel](https://www.huurcommissie.nl/huurcommissie-helpt/beleidsboeken_html/waarderingsstelsel-zelfstandige-woonruimte/de-rubrieken-van-het-woningwaarderingsstelsel-zelfstandige-woning#anker-11-rubriek-11-punten-voor-de-woz-waarde) van de Huurcommissie kent punten toe op basis van bepaalde eigenschappen van de woning (oppervlakte, verwarming, energieprestatie, etc.). 